In [54]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import pickle, threading
from keras.utils import to_categorical
import linecache
from keras.callbacks import Callback
import re
from collections import Counter, defaultdict


epochs = 50
trainable=True

In [2]:

sys.version_info[0]

3

In [3]:
class WeightsSaver(Callback):
    def __init__(self, model, N):
        self.model = model
        self.N = N
        self.batch = 0

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            name = 'german_lm.h5'
#             print("model saved %s" %self.batch)
            self.model.save_weights(name)
        self.batch += 1

In [4]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [5]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [6]:
def twitter_tokenizer(textline:'takes line as an input'):
    
    textline = re.sub('@[\\w_]+', 'USER_MENTION', textline) # replaces @username with 'usermention'
    textline = re.sub('\\|LBR\\|', '', textline) # to replace linebreak
    textline = re.sub('\\.\\.\\.+', '...', textline) #????????
    textline = re.sub('!!+', '!!', textline)#?????????
    textline = re.sub('\\?\\?+', '??', textline)
    words = re.compile('[\\U00010000-\\U0010ffff]|[\\w-]+|[^ \\w\\U00010000-\\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    return(words)

In [ ]:
# creating the first 1m sentences for use
def create_train_data(filename:'name of original file', n_lines:'no of lines to use for training from the twitter data',
                     new_file_name:'name of file to create'):
    new_file = open(new_file_name,'wb')
    n_lines_read=0
    with open(filename,'rb') as f:
        for line in f:
            if n_lines_read == n_lines:
                break
            line = line.decode()
            ids, text = line.split('\t')
            n_lines_read+=1
            text = twitter_tokenizer(text)
            to_write = ' '.join(text)
            to_write+='\n'
#             print(to_write)
            new_file.write(to_write.encode())
            
#             assert n_lines_read<=10
    f.close()
    
            

In [ ]:
create_train_data('/home/jindal/notebooks/twitter_data/twitter-2011_de', 1000000, 
                  '/home/jindal/notebooks/jindal/NER/language_model/training_lines_twitter')

In [ ]:
window_size =51

In [ ]:
# converts the sentences to a fixed size of 51
def generate_sequences(out_file, in_file ) -> None:
    output = open(out_file,'wb')
    with open(in_file,'rb') as f:
        for line in f:
            x+=1
            text = line.decode().split()
            for i in range(len(text)):
                if i+1 >= window_size:
                    temp = text[i-window_size+1:i+1]
                else:
                    temp = ['0' for i in range(window_size - (i +1))] + text[:i+1]
                string = ' '.join(temp)+' \n'
                output.write(string.encode())
    output.close()

In [ ]:
generate_sequences('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_one_million',
                   '/home/jindal/notebooks/jindal/NER/language_model/training_lines_twitter')

In [ ]:
# assert
with open('twitter_dataset_sequences_one_million') as f:
    for line in f:
        assert len(line.split()) == 51

In [7]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [8]:
print(caseEmbeddings.shape)

(8, 8)


In [79]:
characters={}

with open ('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_one_million') as f:
    for line in f:
        for word in line:
            word =str(word)
            for char in word:
#                 print(char)
                characters[char]=True

In [80]:
print(characters)

{'т': True, '\ue410': True, '\ue018': True, '¾': True, 'Đ': True, '♔': True, 'რ': True, '\ue401': True, 'у': True, '\ue224': True, 'һ': True, '\ue115': True, '安': True, '↓': True, '\ue05a': True, '🐟': True, 'Ι': True, '★': True, 'Î': True, '\ue147': True, 'Û': True, '♋': True, '\ue005': True, '└': True, 'K': True, '¸': True, 'ツ': True, '👌': True, '\ue40f': True, '↗': True, 'ײ': True, '›': True, '\u200f': True, 'р': True, '\ue50b': True, '○': True, 'ę': True, '\ue21f': True, '∫': True, '\ue529': True, '\ue502': True, '/': True, '\ue023': True, '❤': True, '☞': True, '➨': True, '╦': True, '➋': True, '\x9e': True, '❋': True, '@': True, 'f': True, '♦': True, 'ő': True, '❍': True, '\ue03c': True, 'ِ': True, 'x': True, 'ò': True, 'Γ': True, '^': True, '-': True, 'ˆ': True, '\ue24d': True, '❥': True, '✋': True, '.': True, '\ue122': True, '聖': True, '▪': True, '&': True, '👼': True, "'": True, '\ue13e': True, '̐': True, '\ue052': True, '\ue695': True, '!': True, '✖': True, 'ɔ': True, '¶': True, 

In [10]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
# print(char2Idx)

In [66]:
# returns the vocab of a file along with a counter of the vocab
def get_vocab(infile:"inputfile"):
    toy_vocab ={}
    toy_vocab_counter = defaultdict(int)


    with open(infile) as f:
        for line in f:
    #         print(line)
            line = line.split()
    #         print(len(line))
            for word in line:
    #             word = word
    #             print(word)
                toy_vocab[word]=True
                toy_vocab_counter[word]+=1
    return toy_vocab, toy_vocab_counter

In [67]:
vocab, freq = get_vocab('/home/jindal/notebooks/jindal/NER/language_model/training_lines_twitter')
most_common_words = sorted(freq, key=freq.get, reverse=True)[:100000]

In [69]:
print(most_common_words[0])

USER_MENTION


In [72]:
def replace_less_frequent(inpfile, outfile):
    outfile = open(outfile,'w')
    with open(inpfile) as f:
        for line in f:
            temp = []
            for word in line.split():
                if word in most_common_words:
                    temp.append(word)
                else:
                    temp.append('NAN')
            string = ' '.join(temp)
            string+='\n'
            outfile.write(string)

In [73]:
replace_less_frequent('training_lines_twitter', 'twitter_1m_pruned_lines')

KeyboardInterrupt: 

In [74]:
toy_vocab, freq = get_vocab('twitter_dataset_sequences_one_million')

In [75]:
line = linecache.getline('/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_one_million', 19)

print(line)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 Terharuan sih aku hehe 



In [76]:
word2Idx={}
for word in toy_vocab.keys():
    word2Idx[word] = len(word2Idx)

In [77]:
word2Idx['und']

874053

In [78]:
print(len(most_co))

474

In [ ]:
print(word2Idx)

In [16]:
line_number=0
# lock = threading.Lock()

In [17]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [98]:
def my_generator(file :"input training file", batch_size, total_lines):
    
    
    global line_number, lock
        
    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        output_labels = []

        for i in range(batch_size):

            index = line_number%total_lines +1
#             print(index)
            line_number+=1
#             lock.release()
#             index = random.choice(len(features), 1)[0]
#             print(index)
            line = linecache.getline(file, index)
            line = line.split()
#             print(line)
            temp_casing = []
            temp_char=[]
            temp_word=[]
#             print(line)
            if len(line)!=51:
                print(index)
                continue
            for word in line[:-1]:
#                 print(word)
                casing =getCasing(word, case2Idx)
#                 print(casing)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
#                 print(temp_char2)
#                 temp_char2 = pad_sequences(temp_char2, 52, padding='post')
#                 temp_char.append(pad_sequences(temp_char2, 52, padding='post'))
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
#             print(temp_word)
#             print(len(temp_word))
#             print(temp_casing)
#             print(temp_char)
#             print(len(temp_char))
#             print(" **************** ")
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(ft.get_word_vector(line[-1]))
#             batch_features[i] = process_features(line, window_size-1, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
#             batch_labels[i] = labels[index]
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

In [ ]:
for inp, output in my_generator('toy_dataset_sequences_shuffled',10):
    continue

In [83]:
vocab_size = len(word2Idx.keys())+1
print(len(most_common_words))

100000


In [31]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [99]:
words_input = Input(shape=(None, 300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(300, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
model.compile(loss='cosine_proximity', optimizer='nadam', metrics=['accuracy'])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 34200       char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_19 (TimeDistri (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [102]:
line_number=0
file_name = '/home/jindal/notebooks/jindal/NER/language_model/twitter_dataset_sequences_one_million'
try:
    model.fit_generator(my_generator(file_name, 64, file_len(file_name)), epochs=1, 
                        steps_per_epoch=file_len(file_name)//64, callbacks=[WeightsSaver(model, 20000)])
except Exception as e:
    model.save('german_lm_twitter.h5')
model.save('german_lm_twitter.h5')

Epoch 1/1
  2858/375540 [..............................] - ETA: 24:59:29 - loss: -0.3447 - acc: 0.1259

KeyboardInterrupt: 